# Импорт библиотек

In [75]:
from PIL import Image, ImageFont
from pilmoji import Pilmoji
import numpy as np
import pickle

# Методы для создания словарей смайликов

In [76]:
def create_emoji_dict(emojis, 
                      background_color=None,
                      emoji_resolution=100, 
                      disp=True):
    """
    Создать словарь смайликов, где каждому смайлику
    соответствует подходящий ему цвет и доля занимаемой
    им площади

    PARAMETERS
    ----------
    emojis : list[str]
        | список из смайликов, из которых создается 
          словарь
    
    background_color : (int, int, int) или None
        | фоновый цвет для смайликов в формате RGB, 
          если None, используется прозрачный фон
    
    emoji_resolution : int
        | каждый смайлик рассматривается как квадрат,
          этот параметр задает размер стороны квадрата.
          Влияет на точность определения среднего цвета
          смайлика, рекомендуется просто оставить 100
    
    disp : bool
        | выводить ли прогресс создания словаря
          (шкала в процентах)
    """
    emoji_dict = {}

    # Параметры для изображения смайликов
    if background_color is None:
        img_back = (0, 0, 0, 0)
    else:
        img_back = (*background_color, 255)
    img_size = (emoji_resolution, emoji_resolution)

    # Цикл по всем смайликам
    for i, emoji in enumerate(emojis, 1):
        # Создание новой картинки со смайликом
        with Image.new('RGBA', img_size, img_back) as image:
            # Шрифт для отображения смайлика
            font = ImageFont.truetype('fonts/arial.ttf', emoji_resolution)
            # Рисование смайлика на картинке
            with Pilmoji(image) as pilmoji:
                pilmoji.text((0, 0), emoji, font=font)

        # Вычисление среднего цвета получившейся картинки смайлика
        if background_color is None:
            # Средний цвет только по непрозрачным пикселям
            np_image = np.array(image)
            weights = np_image[:, :, 3:] / 255
            avg_color = (np_image[:, :, :3] * weights).sum(axis=(0, 1)) / weights.sum()
            # Доля занимаемой смайликом площади
            area = weights.sum() / (emoji_resolution ** 2)
        else:
            # Средний цвет по каналам RGB
            avg_color = np.array(image)[:, :, :3].mean(axis=(0, 1))
            area = 1

        # Добавление смайлика в словарь
        emoji_dict[emoji] = (avg_color, area)

        # Вывод прогресса
        if disp:
            progress = round(i / len(emojis) * 100, 2)
            print(f'\rВыполнено: {progress:>6}% ({i}/{len(emojis)})', end='')

    if disp:
        print()

    return emoji_dict

In [77]:
def load_emojis(name):
    """
    Загрузить список из смайликов {name}, лежащий в
    папке emoji_sources/
    """
    with open(f'emoji_sources/{name}.txt', 'r') as file:
        emoji_list = [line.strip() for line in file]
    return emoji_list

# Создание словаря смайликов

In [94]:
# Название нового словаря смайликов
emoji_dict_name = 'moon_black'
# Источник смайликов (файл из папки emoji_sources/)
emoji_source = 'moon'
# Фоновый цвет смайликов
background_color = (0, 0, 0)

In [95]:
%%time

# Создание словаря
emoji_dict = create_emoji_dict(emojis=load_emojis(emoji_source), 
                               background_color=background_color)

# Сохранение словаря смайликов
with open(f'emoji_dicts/{emoji_dict_name}.pkl', 'wb') as file:
    pickle.dump(emoji_dict, file)

# Вывод результатов
print(f'Словарь {emoji_dict_name} создан и сохранен:')
print(''.join(s for s in list(emoji_dict.keys())[:10]) + '...\n')

Выполнено:  100.0% (8/8)
Словарь moon_black создан и сохранен:
🌕🌖🌗🌘🌑🌒🌓🌔...

CPU times: user 855 ms, sys: 32.1 ms, total: 887 ms
Wall time: 5.35 s
